# Mass

### Import

In [1]:
import numpy as np
import pandas as pd
import math 

from scipy.spatial import distance

### Define algorithm

In [2]:
def alg(_input):
    i_x = 0
    i_y = 1
    i_tss = 5
    
    total_mass = 0
    total_mass_x = 0
    total_mass_y = 0
    
    for anchor in _input:
        anchor_mass = math.sqrt(anchor[i_tss])
        total_mass += anchor_mass
        total_mass_x += (anchor_mass * anchor[i_x])
        total_mass_y += (anchor_mass * anchor[i_y])
    
    pos_x = total_mass_x / total_mass
    pos_y = total_mass_y / total_mass
    
    return (pos_x, pos_y)

### Create sections

In [3]:
def create_sections():
    windows_size = 7
    sections = []

    width = 8
    for x in [0, 12]:
        for y in range(0,69,windows_size):
            height = 7 if y != 63 else 9
            sections.append([(x,y), (x+width, y+height)])

    sections = list(reversed(sections))

    for i in range(len(sections)):
        print("Section {:02d}: from point ({:02d}, {:02d}) to point ({:02d}, {:02d})".format(i+1, sections[i][0][0], sections[i][0][1], sections[i][1][0], sections[i][1][1]))

    return sections, windows_size

sections_xy, windows_size = create_sections()

Section 01: from point (12, 63) to point (20, 72)
Section 02: from point (12, 56) to point (20, 63)
Section 03: from point (12, 49) to point (20, 56)
Section 04: from point (12, 42) to point (20, 49)
Section 05: from point (12, 35) to point (20, 42)
Section 06: from point (12, 28) to point (20, 35)
Section 07: from point (12, 21) to point (20, 28)
Section 08: from point (12, 14) to point (20, 21)
Section 09: from point (12, 07) to point (20, 14)
Section 10: from point (12, 00) to point (20, 07)
Section 11: from point (00, 63) to point (08, 72)
Section 12: from point (00, 56) to point (08, 63)
Section 13: from point (00, 49) to point (08, 56)
Section 14: from point (00, 42) to point (08, 49)
Section 15: from point (00, 35) to point (08, 42)
Section 16: from point (00, 28) to point (08, 35)
Section 17: from point (00, 21) to point (08, 28)
Section 18: from point (00, 14) to point (08, 21)
Section 19: from point (00, 07) to point (08, 14)
Section 20: from point (00, 00) to point (08, 07)


### Section utils

In [4]:
def xy_to_section(pos_x, pos_y):
    pos_x = 8 if (8 < pos_x <= 10) else pos_x
    pos_x = 12 if (10 < pos_x <= 12) else pos_x
    
    for i, section in enumerate(sections_xy):
        if section[0][0] <= pos_x <= section[1][0] and section[0][1] <= pos_y <= section[1][1]:
            return i
    
    print("Error: no section")

def section_distance(true, predicted):
    true = sections_xy[true]
    predicted = sections_xy[predicted]

    if true[0][0] == predicted[0][0]:
        res = abs(true[0][1]/windows_size - predicted[0][1]/windows_size)
    else:
        if true[0][1] == predicted[0][1]:
            res = 1
        else:
            res = abs(true[0][1]/windows_size - predicted[0][1]/windows_size)
            
    return res 

### Run functions

In [5]:
def get_data():
    header_list = ["cowshed", "cow", "true_pos_x", "true_pos_y", "anchor_x", "anchor_y", "anchor_id", "rssi", "rssi_std", "samples", "tss"]

    df_all_grouped = pd.read_csv("./data.csv", names=header_list, sep=";").groupby(["cowshed", "cow", "true_pos_x", "true_pos_y"])
    df_all_grouped = [df_all_grouped.get_group(x) for x in df_all_grouped.groups]
    
    data = []
    for df in df_all_grouped:
        df = df.sort_values(by="samples", ascending=False)
        df = df.drop_duplicates(subset=["anchor_id"])
        
        if not(len(df) >= 6 and len(df.loc[df["samples"] > 18]) >= 4):
            continue
    
        _input = pd.DataFrame(df, columns=["anchor_x", "anchor_y", "rssi", "rssi_std", "samples", "tss"]).to_numpy()
        _ground_truth = tuple(pd.DataFrame(df, columns=["true_pos_x", "true_pos_y"]).to_numpy()[0])
        
        data.append((_input, _ground_truth))
    return data

def print_results(errors, pre_text = ''):
    e = np.array(errors)
    e = np.sort(e)
    _min = round(e.min(), 2)
    _avg = round(e.mean(), 2)
    _std = round(e.std(), 2)
    _median = round(np.percentile(e, 50, interpolation='nearest'), 2)
    _per90 = round(np.percentile(e, 90, interpolation='nearest'), 2)
    _per95 = round(np.percentile(e, 95, interpolation='nearest'), 2)
    _max = round(e.max(), 2)
    print("{:15s} min = {:.2f}, avg = {:.2f}, std = {:.2f}, median = {:.2f}, 90 pctil = {:.2f}, 95 pctil = {:.2f}, max = {:.2f}".format(pre_text, _min, _avg, _std, _median, _per90, _per95, _max))

def run():
    data = get_data()
    
    errors_xy = []
    errors_sections = []    
    for (_input, _ground_truth) in data:
        predicted = alg(_input)
        
        xy_error = distance.euclidean(_ground_truth, predicted)
        errors_xy.append(xy_error)
        
        true_section = xy_to_section(*_ground_truth)
        predicted_section = xy_to_section(*predicted)
        section_error = section_distance(true_section, predicted_section)
        errors_sections.append(section_error)

    print_results(errors_xy, "XY error:")
    print_results(errors_sections, "Sections error:")

### Run

In [6]:
run()

XY error:       min = 0.60, avg = 8.85, std = 4.63, median = 8.65, 90 pctil = 14.87, 95 pctil = 15.57, max = 25.45
Sections error: min = 0.00, avg = 1.07, std = 0.66, median = 1.00, 90 pctil = 2.00, 95 pctil = 2.00, max = 3.00
